A couple of people have recently asked me about how to use [scikit-allel](http://scikit-allel.readthedocs.org) to select data from a variant call set for a particular set of variants. This could be variants for a specific genome region (e.g., a gene), or variants matching a particular set of filters. This notebook gives a couple of examples, using data from the (human) 1000 genomes project phase 3.

Here's the Python packages we'll need. If you have earlier versions, you'll need to upgrade.

In [1]:
import allel
allel.__version__

'1.1.10'

In [2]:
import zarr
zarr.__version__

'2.2.0'

In [3]:
import numcodecs
numcodecs.__version__

'0.5.5'

In [4]:
import numpy as np
np.__version__

'1.13.3'

In [5]:
# other imports
import sys

## Extract data from a VCF

The source data comes as a VCF file, which I've downloaded to the local disk:

In [6]:
vcf_path = 'data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz'
!ls -lh {vcf_path}

-rw-r--r-- 1 aliman aliman 205M Jun 20  2017 data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz


I'm going to use data from chromosome 22 only for illustration. I'm also going to extract the data and convert to Zarr format, which will make life easier downstream. To do the conversion I'm going to use the [vcf_to_zarr()](http://scikit-allel.readthedocs.io/en/latest/io.html#allel.vcf_to_zarr) function from scikit-allel. This is a one-off operation, once the data have been converted to Zarr they can be loaded directly from Zarr the next time you want to do some analysis. The conversion takes about 3 minutes on my computer.

In [7]:
zarr_path = 'data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.zarr'

In [8]:
allel.vcf_to_zarr(vcf_path, zarr_path, group='22', fields='*', log=sys.stdout, overwrite=True)

[vcf_to_zarr] 65536 rows in 11.57s; chunk in 11.57s (5664 rows/s); 22 :18539397
[vcf_to_zarr] 131072 rows in 24.17s; chunk in 12.60s (5200 rows/s); 22 :21016127
[vcf_to_zarr] 196608 rows in 35.83s; chunk in 11.66s (5620 rows/s); 22 :23236362
[vcf_to_zarr] 262144 rows in 48.94s; chunk in 13.11s (4998 rows/s); 22 :25227844
[vcf_to_zarr] 327680 rows in 60.08s; chunk in 11.13s (5886 rows/s); 22 :27285434
[vcf_to_zarr] 393216 rows in 71.06s; chunk in 10.99s (5965 rows/s); 22 :29572822
[vcf_to_zarr] 458752 rows in 82.07s; chunk in 11.01s (5955 rows/s); 22 :31900536
[vcf_to_zarr] 524288 rows in 93.09s; chunk in 11.02s (5946 rows/s); 22 :34069864
[vcf_to_zarr] 589824 rows in 104.20s; chunk in 11.11s (5899 rows/s); 22 :36053392
[vcf_to_zarr] 655360 rows in 115.38s; chunk in 11.18s (5861 rows/s); 22 :38088395
[vcf_to_zarr] 720896 rows in 126.86s; chunk in 11.48s (5708 rows/s); 22 :40216200
[vcf_to_zarr] 786432 rows in 139.49s; chunk in 12.63s (5189 rows/s); 22 :42597446
[vcf_to_zarr] 851968 rows

Let's open the Zarr data and do a little bit of poking around to see what's there.

In [9]:
callset = zarr.open_group(zarr_path, mode='r')
callset.tree(expand=True)

/
 └── 22
     ├── calldata
     │   └── GT (1103547, 2504, 2) int8
     ├── samples (2504,) object
     └── variants
         ├── AA (1103547,) object
         ├── AC (1103547, 3) int32
         ├── AF (1103547, 3) float32
         ├── AFR_AF (1103547, 3) float32
         ├── ALT (1103547, 3) object
         ├── AMR_AF (1103547, 3) float32
         ├── AN (1103547,) int32
         ├── CHROM (1103547,) object
         ├── CIEND (1103547, 2) int32
         ├── CIPOS (1103547, 2) int32
         ├── CS (1103547,) object
         ├── DP (1103547,) int32
         ├── EAS_AF (1103547, 3) float32
         ├── END (1103547,) int32
         ├── EUR_AF (1103547, 3) float32
         ├── EX_TARGET (1103547,) bool
         ├── FILTER_PASS (1103547,) bool
         ├── ID (1103547,) object
         ├── IMPRECISE (1103547,) bool
         ├── MC (1103547,) object
         ├── MEINFO (1103547, 4) object
         ├── MEND (1103547,) int32
         ├── MLEN (1103547,) int32
         ├── MSTART (1103547,) int32
         ├── MULTI_ALLELIC (1103547,) bool
         ├── NS (1103547,) int32
         ├── POS (1103547,) int32
         ├── QUAL (1103547,) float32
         ├── REF (1103547,) object
         ├── SAS_AF (1103547, 3) float32
         ├── SVLEN (1103547,) int32
         ├── SVTYPE (1103547,) object
         ├── TSD (1103547,) object
         ├── VT (1103547,) object
         ├── is_snp (1103547,) bool
         ├── numalt (1103547,) int32
         └── svlen (1103547, 3) int32

The `tree()` method shows us how the data are organised hierarchically within the Zarr store. Hopefully you can see that there is a root group indicated by a slash ('/'), then below that there is a group named '22' containing all of the data from Chromosome 22, then there is a group called 'calldata' and another called 'variants'. 

Within the 'calldata' group there is an array named 'GT' which has the genotypes.

Within the 'variants' group there are arrays named 'CHROM', 'POS', 'DP', etc., containing information about the variants that have been called.

Let's get some diagnostics on the Zarr genotype array.

In [10]:
gt_zarr = callset['22/calldata/GT']
gt_zarr.info

Name,/22/calldata/GT
Type,zarr.core.Array
Data type,int8
Shape,"(1103547, 2504, 2)"
Chunk shape,"(65536, 64, 2)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,5526563376 (5.1G)
No. bytes stored,293489697 (279.9M)


This tells us the uncompressed size of the genotype array is 5.1 gigabytes. The actual size on disk is smaller (279.9 megabytes) because the data are compressed. 

## Loading data for a gene

If you want to work with data from a single gene, or any other contiguous region within a chromosome, here's how to do it.

First you need the positions of the variants, wrapped as a scikit-allel ``SortedIndex``:

In [11]:
pos = allel.SortedIndex(callset['22/variants/POS'])
pos

0,1,2,3,4,...,1103542,1103543,1103544,1103545,1103546
16050075,16050115,16050213,16050319,16050527,...,51241342,51241386,51244163,51244205,51244237


The numbers at the top are the variant indices (starting from 0). The numbers at the bottom are the variant positions as genomic coordinates (i.e., number of base pairs from the chromosome start, starting from 1).

To extract data for a chromosome region, you can use the `pos` object to translate genomic coordinates into variant indices. For example, if you want to locate data for the region from position 20,000,000 to 20,100,000, you can do this:

In [12]:
loc_region = pos.locate_range(20000000, 20100000)
loc_region

slice(108029, 111127, None)

The `loc_region` object is a slice, which is simply a way of storing a start and stop index. Here the start index is 108,029 and the stop index is 111,127. So, we need data starting from the 108,030th variant up to but not including the 111,128th variant (remembering that indices start from zero).

We can now use this slice to extract genotypes for our genome region of interest:

In [13]:
gt_region = allel.GenotypeArray(gt_zarr[loc_region])
gt_region

<GenotypeArray shape=(3098, 2504, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

Note that `gt_region` is a genotype array with 3,098 variants and 2,504 samples.

Breaking this down, the ``gt_zarr[loc_region]`` code loads data for the requested slice of the genotype data from disk into memory as a numpy array. For convenience, I have wrapped this numpy array using the ``allel.GenotypeArray`` class, which provides a nicer visual representation of the data and gives some useful methods.

## Filtering variants

Filtering variants is a very common task. Each analysis typically needs a different set of variants to work with. For example, you may need to filter variants based on some metrics of quality, or on other conditions like allele frequency.

When filtering variants, you first need to identify which variants you require. To help with this, I'm first going to load up some variant information.

The 'MULTI_ALLELIC' array in this callset is a Boolean array indicating whether a variants is multi-allelic (has more than one alternate allele) or not. The following code loads this array from disk into memory:

In [14]:
multi_allelic = callset['22/variants/MULTI_ALLELIC'][:]
multi_allelic

array([False, False, False, ..., False, False, False], dtype=bool)

The 'AFR_AF' array in this callset has alternate allele frequencies for the African samples within the cohort:

In [15]:
afr_af = callset['22/variants/AFR_AF'][:]
afr_af

array([[ 0.    ,     nan,     nan],
       [ 0.0234,     nan,     nan],
       [ 0.0272,     nan,     nan],
       ..., 
       [ 0.    ,     nan,     nan],
       [ 0.    ,     nan,     nan],
       [ 0.    ,     nan,     nan]], dtype=float32)

Here the `afr_af` array has multiple columns, one for each alternate allele. 

Let's locate variants that are not multi-allelic and have an African allele frequency above 5%:

In [16]:
loc_selection = ~multi_allelic & (afr_af[:, 0] > 0.05)
loc_selection

array([False, False, False, ..., False, False, False], dtype=bool)

How many variants match our query?

In [18]:
np.count_nonzero(loc_selection)

138275

Now, to extract genotype data for these variants, there are a couple of ways to do it.

If the full genotype array is not too big, you can start by loading the whole lot into memory. In this case, the genotype array is 5.1G, and I have enough RAM on my laptop to handle that, so let's do it:

In [19]:
gt = allel.GenotypeArray(gt_zarr)
gt

<GenotypeArray shape=(1103547, 2504, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

Now to extract genotypes for the selection, you can use the `compress()` method, e.g.:

In [20]:
gt_selection = gt.compress(loc_selection, axis=0)
gt_selection

<GenotypeArray shape=(138275, 2504, 2) dtype=int8>
0/1 0/0 0/0 0/0 0/0 ... 1/0 0/1 0/0 0/0 0/0
0/1 0/0 0/0 1/0 1/0 ... 1/1 0/1 1/1 0/0 1/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 1/0 0/0 0/0 0/0

Alternatively, if your data are larger and/or your computer doesn't have much RAM, there is another way to do this. This makes use of a Python package called Dask, which allows you to run computations without loading all data into memory. To use Dask, we can start by wrapping the full genotype array with the `allel.GenotypeDaskArray` class:

In [21]:
gt_dask = allel.GenotypeDaskArray(gt_zarr)
gt_dask

<GenotypeDaskArray shape=(1103547, 2504, 2) dtype=int8>

Now we can apply the selection, using almost the same syntax, except that when working via Dask we need to call the `compute()` method to get the final result:

In [22]:
gt_selection = gt_dask.compress(loc_selection, axis=0).compute()
gt_selection

<GenotypeArray shape=(138275, 2504, 2) dtype=int8>
0/1 0/0 0/0 0/0 0/0 ... 1/0 0/1 0/0 0/0 0/0
0/1 0/0 0/0 1/0 1/0 ... 1/1 0/1 1/1 0/0 1/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 1/0 0/0 0/0 0/0

## Further reading

Hopefully these examples have been helpful. For further information and more examples, the following may be useful:

* [Extracting data from VCF files](http://alimanfoo.github.io/2017/06/14/read-vcf.html)
* [A tour of scikit-allel](http://alimanfoo.github.io/2016/06/10/scikit-allel-tour.html) (N.B., this is an older article and uses slightly different techniques from the ones used here, but still relevant.)
